### Problem Statement:
Predict the presence or absence of a sleep disorder (Insomnia, Sleep Apnea, or None) based on lifestyle and health factors such as age, gender, quality of sleep, physical activity level, stress levels, BMI category, and blood pressure.

### Here's a brief outline of how I tackled this project:

#### Data Preprocessing:
I initially cleaned the data by transforming categorical variables to numerical ones where necessary (for instance, transforming 'Gender' and 'BMI Category' using Label Encoding) and by extracting systolic and diastolic blood pressure as separate features from the 'Blood Pressure' column. I then selected relevant features for predicting sleep disorders.

#### Handling Imbalanced Data:
I noticed a class imbalance in the target variable 'Sleep Disorder'. To handle this, I used Synthetic Minority Over-sampling Technique (SMOTE) to create a balanced dataset by oversampling the minority class. This made the model more capable of distinguishing between different classes.

#### Model Selection & Training:
I chose to use the Random Forest Classifier due to its robustness and versatility. This model was trained on the resampled dataset.

#### Model Evaluation:
I used the standard train-test-split approach for validating the model. The model's predictions were evaluated based on the accuracy achieved on the test dataset.

#### Saving the Model:
I serialized the trained model using joblib, allowing it to be saved to a file and then reloaded later. This is an important step for deploying the model.

#### Creating an Interactive Predictive Tool:
I created a simple, interactive tool using the ipywidgets library. This tool allows a user to input their personal data, and then it uses the trained model to predict whether the user is likely to have a sleep disorder.

While I didn't deploy this model as a web application or dive deep into model interpretability for this project, these are potential next steps that could be taken. These additional steps would allow me to showcase more skills in machine learning engineering and explain the decision-making process of the model, respectively.

Throughout this process, I was able to apply and demonstrate skills in various important areas of a machine learning project, including data preprocessing, handling imbalanced datasets, model selection and training, and creating an interactive tool for making predictions.

# Dataset and Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv')
df

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,None
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
370,371,Female,59,Nurse,8.0,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
371,372,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
372,373,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea


# Exploratory Data Analysis

In [3]:
df.isnull().sum()

Person ID                  0
Gender                     0
Age                        0
Occupation                 0
Sleep Duration             0
Quality of Sleep           0
Physical Activity Level    0
Stress Level               0
BMI Category               0
Blood Pressure             0
Heart Rate                 0
Daily Steps                0
Sleep Disorder             0
dtype: int64

Great! no missing or null values

In [4]:
df['Sleep Disorder'].value_counts()

None           219
Sleep Apnea     78
Insomnia        77
Name: Sleep Disorder, dtype: int64

Definitely can see class imbalances

In [5]:
df['Occupation'].unique()

array(['Software Engineer', 'Doctor', 'Sales Representative', 'Teacher',
       'Nurse', 'Engineer', 'Accountant', 'Scientist', 'Lawyer',
       'Salesperson', 'Manager'], dtype=object)

In [6]:
software_engineer_mean = df[df['Occupation'] == 'Software Engineer']['Sleep Duration'].mean()
engineer_mean = df[df['Occupation'] == 'Engineer']['Sleep Duration'].mean()
print("Mean Sleep Duration for Software Engineers: ", software_engineer_mean)
print("Mean Sleep Duration for Engineers: ", engineer_mean)

Mean Sleep Duration for Software Engineers:  6.75
Mean Sleep Duration for Engineers:  7.987301587301586


Software Engineers and Engineers are Not similiar

In [7]:
Doctor_mean = df[df['Occupation'] == 'Doctor']['Sleep Duration'].mean()
Nurse_mean = df[df['Occupation'] == 'Nurse']['Sleep Duration'].mean()

print("Mean Sleep Duration for Doctor: ", Doctor_mean)
print("Mean Sleep Duration for Nurse: ", Nurse_mean)

Mean Sleep Duration for Doctor:  6.970422535211269
Mean Sleep Duration for Nurse:  7.0630136986301375


The Nurse and Doctor are Extermely similiar

In [8]:
# Combining both Doctor and Nurse due to how similiar they are
df['Occupation'] = df['Occupation'].replace(['Doctor', 'Nurse'], 'Healthcare')

In [9]:
# BMI Category
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['BMI Category'] = le.fit_transform(df['BMI Category'])

In [10]:
# Gender Male: 1 and Female: 0
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])

In [11]:
df = pd.get_dummies(df, columns=['Occupation'])

In [12]:
df[['Systolic_BP', 'Diastolic_BP']] = df['Blood Pressure'].str.split('/', expand=True)
df['Systolic_BP'] = pd.to_numeric(df['Systolic_BP'])
df['Diastolic_BP'] = pd.to_numeric(df['Diastolic_BP'])
df = df.drop(columns='Blood Pressure')

In [13]:
display(df)

,Person ID,Gender,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,...,Occupation_Healthcare,Occupation_Lawyer,Occupation_Manager,Occupation_Sales Representative,Occupation_Salesperson,Occupation_Scientist,Occupation_Software Engineer,Occupation_Teacher,Systolic_BP,Diastolic_BP
0,1,1,27,6.1,6,42,6,3,77,4200,...,0,0,0,0,0,0,1,0,126,83
1,2,1,28,6.2,6,60,8,0,75,10000,...,1,0,0,0,0,0,0,0,125,80
2,3,1,28,6.2,6,60,8,0,75,10000,...,1,0,0,0,0,0,0,0,125,80
3,4,1,28,5.9,4,30,8,2,85,3000,...,0,0,0,1,0,0,0,0,140,90
4,5,1,28,5.9,4,30,8,2,85,3000,...,0,0,0,1,0,0,0,0,140,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,0,59,8.1,9,75,3,3,68,7000,...,1,0,0,0,0,0,0,0,140,95
370,371,0,59,8.0,9,75,3,3,68,7000,...,1,0,0,0,0,0,0,0,140,95
371,372,0,59,8.1,9,75,3,3,68,7000,...,1,0,0,0,0,0,0,0,140,95
372,373,0,59,8.1,9,75,3,3,68,7000,...,1,0,0,0,0,0,0,0,140,95


# ML Modeling

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X = df.drop('Sleep Disorder', axis=1)
y = df['Sleep Disorder']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
# print(x)
print(y_pred)
# print(classification_report(y_test, y_pred))

['None' 'None' 'None' 'None' 'None' 'Insomnia' 'None' 'None' 'None' 'None'
 'Insomnia' 'None' 'None' 'None' 'None' 'Sleep Apnea' 'None' 'Insomnia'
 'None' 'None' 'None' 'None' 'None' 'Insomnia' 'Insomnia' 'None'
 'Insomnia' 'Insomnia' 'Insomnia' 'Sleep Apnea' 'None' 'None' 'None'
 'Sleep Apnea' 'Sleep Apnea' 'None' 'None' 'None' 'None' 'None' 'Insomnia'
 'Sleep Apnea' 'Insomnia' 'Insomnia' 'None' 'None' 'None' 'None'
 'Sleep Apnea' 'None' 'Sleep Apnea' 'Insomnia' 'Sleep Apnea' 'None'
 'Insomnia' 'Sleep Apnea' 'Insomnia' 'Sleep Apnea' 'Sleep Apnea' 'None'
 'None' 'Insomnia' 'None' 'None' 'Insomnia' 'None' 'Insomnia' 'None'
 'None' 'None' 'None' 'Insomnia' 'None' 'Sleep Apnea' 'Sleep Apnea']


In [15]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

clf_resampled = RandomForestClassifier(random_state=42)
clf_resampled.fit(X_train_resampled, y_train_resampled)

# print(X_test_resampled)
y_pred_resampled = clf_resampled.predict(X_test_resampled)

print(classification_report(y_test_resampled, y_pred_resampled))

              precision    recall  f1-score   support

    Insomnia       0.84      0.90      0.87        29
        None       0.94      0.96      0.95        52
 Sleep Apnea       0.96      0.90      0.93        51

    accuracy                           0.92       132
   macro avg       0.91      0.92      0.92       132
weighted avg       0.93      0.92      0.92       132



The SMOTE (Synthetic Minority Over-sampling Technique) is an oversampling technique used to balance the classes in an imbalanced dataset

# LazyPredict

In [16]:
!pip install lazypredict

In [17]:
from lazypredict.Supervised import LazyClassifier

In [18]:
# Creating an instance of LazyClassificer
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

display(models)

 83%|████████▎ | 24/29 [00:01<00:00, 19.27it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000278 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 257
[LightGBM] [Info] Number of data points in the train set: 299, number of used features: 18
[LightGBM] [Info] Start training from score -1.589570
[LightGBM] [Info] Start training from score -0.529960
[LightGBM] [Info] Start training from score -1.573309
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

100%|██████████| 29/29 [00:01<00:00, 17.23it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
Perceptron,0.92,0.89,None,0.92,0.02
RidgeClassifierCV,0.91,0.88,None,0.91,0.02
BaggingClassifier,0.92,0.88,None,0.92,0.04
LinearSVC,0.91,0.87,None,0.91,0.09
CalibratedClassifierCV,0.91,0.87,None,0.91,0.24
RidgeClassifier,0.89,0.86,None,0.89,0.03
DecisionTreeClassifier,0.91,0.85,None,0.90,0.01
LinearDiscriminantAnalysis,0.88,0.85,None,0.88,0.04
SGDClassifier,0.89,0.85,None,0.89,0.02


In [19]:
# Creating an instance of LazyClassificer on resampled data
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled)

display(models)

 86%|████████▌ | 25/29 [00:02<00:00, 14.17it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000139 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 386
[LightGBM] [Info] Number of data points in the train set: 525, number of used features: 18
[LightGBM] [Info] Start training from score -1.016374
[LightGBM] [Info] Start training from score -1.145404
[LightGBM] [Info] Start training from score -1.139434
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur

100%|██████████| 29/29 [00:02<00:00, 12.27it/s]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearSVC,0.94,0.93,None,0.94,0.18
ExtraTreesClassifier,0.93,0.93,None,0.93,0.22
LogisticRegression,0.93,0.92,None,0.93,0.06
CalibratedClassifierCV,0.93,0.92,None,0.93,0.42
RandomForestClassifier,0.92,0.92,None,0.92,0.25
LabelPropagation,0.92,0.92,None,0.92,0.06
LabelSpreading,0.92,0.92,None,0.92,0.13
SGDClassifier,0.91,0.91,None,0.91,0.06
SVC,0.90,0.91,None,0.90,0.04


In [20]:
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', LinearSVC(random_state=42))
])

param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__tol': [10 ,1 ,0.1 ,1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
}

# A GridSearchCV object
grid_search = GridSearchCV(pipe, param_grid, cv=5)
grid_search.fit(X_train_resampled, y_train_resampled)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
y_pred = grid_search.predict(X_test_resampled)
print(classification_report(y_test_resampled, y_pred, target_names=['Insomnia', 'None', 'Sleep Apnea']))

              precision    recall  f1-score   support

    Insomnia       0.93      0.86      0.89        29
        None       0.93      0.96      0.94        52
 Sleep Apnea       0.96      0.96      0.96        51

    accuracy                           0.94       132
   macro avg       0.94      0.93      0.93       132
weighted avg       0.94      0.94      0.94       132



In [21]:
print(best_params)

{'svc__C': 1, 'svc__tol': 0.1}


# Next Steps

In [22]:
importances = clf_resampled.feature_importances_
feature_names = X_train_resampled.columns
feature_importances = dict(zip(feature_names, importances))
sorted_feature_importances = sorted(feature_importances.items(), key=lambda item: item[1], reverse=True)
print(sorted_feature_importances)

[('Person ID', 0.14852511893508102), ('Diastolic_BP', 0.14774320115962555), ('Systolic_BP', 0.1357872439054357), ('BMI Category', 0.10920245722519521), ('Physical Activity Level', 0.08884135583779726), ('Sleep Duration', 0.07717184415852316), ('Daily Steps', 0.07266054403137205), ('Age', 0.05871879361692078), ('Occupation_Healthcare', 0.03510059529776193), ('Quality of Sleep', 0.031469457002867754), ('Heart Rate', 0.02949396149723242), ('Stress Level', 0.02298108401982315), ('Occupation_Salesperson', 0.010910854775814516), ('Gender', 0.009248844391415139), ('Occupation_Lawyer', 0.006030525152674882), ('Occupation_Teacher', 0.005219293024668201), ('Occupation_Engineer', 0.004721041493003322), ('Occupation_Accountant', 0.0030116539009941), ('Occupation_Software Engineer', 0.0012436514904302484), ('Occupation_Sales Representative', 0.0012383882464336356), ('Occupation_Scientist', 0.0004681072613003495), ('Occupation_Manager', 0.0002119835756295594)]


In [23]:
!pip install eli5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107719 sha256=1314e2360beb42e1eaa6dffe31f71005733854f6190bf80c06c66843dc7b7c9f
  Stored in directory: /root/.cache/pip/wheels/b8/58/ef/2cf4c306898c2338d51540e0922c8e0d6028e07007085c0004
Successfully built eli5


In [24]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(clf_resampled, random_state=1).fit(X_test_resampled, y_test_resampled)
eli5.show_weights(perm, feature_names = X_test_resampled.columns.tolist())

Weight,Feature
0.1000 ± 0.0113,Person ID
0.0152 ± 0.0096,BMI Category
0.0106 ± 0.0121,Systolic_BP
0.0091 ± 0.0113,Diastolic_BP
0.0076 ± 0.0000,Gender
0.0061 ± 0.0177,Quality of Sleep
0.0030 ± 0.0155,Physical Activity Level
0.0015 ± 0.0113,Stress Level
0 ± 0.0000,Occupation_Teacher
0 ± 0.0000,Occupation_Software Engineer


Importing a tool called Permutation Importance from the library eli5

# Training for Deployment

In [27]:
from imblearn.over_sampling import SMOTE
import joblib
# Load your data
df = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv')
le = LabelEncoder()

df['BMI Category'] = le.fit_transform(df['BMI Category'])
df['Gender'] = le.fit_transform(df['Gender'])
df[['Systolic_BP', 'Diastolic_BP']] = df['Blood Pressure'].str.split('/', expand=True)
df['Systolic_BP'] = pd.to_numeric(df['Systolic_BP'])
df['Diastolic_BP'] = pd.to_numeric(df['Diastolic_BP'])

df = df.drop(columns='Blood Pressure')
features = ['Age', 'BMI Category', 'Systolic_BP', 'Diastolic_BP', 'Gender', 'Quality of Sleep', 'Physical Activity Level', 'Stress Level', 'Sleep Disorder']

df = df[features]
X = df.drop('Sleep Disorder', axis=1)
y = df['Sleep Disorder']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

clf_resampled = RandomForestClassifier(random_state=42)
clf_resampled.fit(X_train_resampled, y_train_resampled)

y_pred_resampled = clf_resampled.predict(X_test_resampled)
joblib.dump(clf_resampled, 'model.pkl')

['model.pkl']

In [28]:
import ipywidgets as widgets
import numpy

this_mode=joblib.load("model.pkl",mmap_mode="r")


input_age = widgets.BoundedIntText(value=25, min=0, max=100, description='Age:', step=1)
input_bmi_category = widgets.BoundedIntText(value=25, min=0, max=50, description='BMI Category:', step=1)
input_systolic_bp = widgets.BoundedIntText(value=120, min=0, max=200, description='Systolic BP:', step=1)
input_diastolic_bp = widgets.BoundedIntText(value=80, min=0, max=200, description='Diastolic BP:', step=1)
input_gender = widgets.Dropdown(options=['Male', 'Female'], value='Male', description='Gender:')
input_quality_of_sleep = widgets.BoundedFloatText(value=7, min=0, max=10, description='Quality of Sleep:', step=0.1)
input_physical_activity_level = widgets.BoundedFloatText(value=5, min=0, max=10, description='Physical Activity Level:', step=0.1)
input_stress_level = widgets.BoundedFloatText(value=5, min=0, max=10, description='Stress Level:', step=0.1)

button_predict = widgets.Button(description='Predict')
def on_button_predict_clicked(b):
        if input_gender.value == "Male":
          input_gender_value=1
        else:
          input_gender_value=0
        this_input= np.array([int(input_age.value),int(input_bmi_category.value),int(input_systolic_bp.value),int(input_diastolic_bp.value),int(input_gender_value),int(input_quality_of_sleep.value),int(input_physical_activity_level.value),int(input_stress_level.value)])
        print(this_input)
        ans=this_mode.predict([this_input])
        print(ans)
button_predict.on_click(on_button_predict_clicked)

display(input_age)
display(input_bmi_category)
display(input_systolic_bp)
display(input_diastolic_bp)
display(input_gender)
display(input_quality_of_sleep)
display(input_physical_activity_level)
display(input_stress_level)
display(button_predict)

BoundedIntText(value=25, description='Age:')

BoundedIntText(value=25, description='BMI Category:', max=50)

BoundedIntText(value=120, description='Systolic BP:', max=200)

BoundedIntText(value=80, description='Diastolic BP:', max=200)

Dropdown(description='Gender:', options=('Male', 'Female'), value='Male')

BoundedFloatText(value=7.0, description='Quality of Sleep:', max=10.0, step=0.1)

BoundedFloatText(value=5.0, description='Physical Activity Level:', max=10.0, step=0.1)

BoundedFloatText(value=5.0, description='Stress Level:', max=10.0, step=0.1)

Button(description='Predict', style=ButtonStyle())

[ 25  25 120  80   1   7   5   5]
['None']
[ 25  25 140  80   1   2   5   7]
['Sleep Apnea']
